# MÓDULO 8: INTRODUCCIÓN A LANGGRAPH
## TEORÍA - SESIÓN 1: Fundamentos de LangGraph y Arquitectura de Grafos

**Objetivo:** Comprender qué es LangGraph, cuándo usarlo, y construir tu primer grafo funcional.

**Estructura:**
- Bloque 1: ¿Qué es LangGraph y cuándo usarlo?
- Bloque 2: Anatomía de un Grafo LangGraph
- Bloque 3: Construyendo el Primer Grafo Completo

Instalación de dependencias necesarias.

In [ ]:
# Instalación
!pip install -q langgraph grandalf

**Nota sobre visualización:**  
A lo largo de este notebook verás visualizaciones de los grafos usando `draw_mermaid_png()`. 
Esto te ayudará a **ver** cómo se conectan los nodos, especialmente útil con grafos condicionales y ciclos.

---
## BLOQUE 1: ¿Qué es LangGraph y cuándo usarlo?

### 1.1 El Problema: Limitaciones de las Cadenas Lineales

En el curso anterior trabajamos con **LangChain**, que nos permitía construir flujos de procesamiento mediante **cadenas** (chains).

**Cadena típica de LangChain:**
```
Prompt → LLM → Parser → Output
    A  →  B  →   C    →   D
```

Este flujo funciona bien para tareas **lineales y predecibles**. Sin embargo, tiene limitaciones:

**Limitación 1: Sin ciclos**  
¿Qué pasa si el output del paso C no es válido y necesitas volver al paso B?

**Limitación 2: Sin decisiones dinámicas**  
¿Qué pasa si dependiendo de la respuesta del LLM necesitas ir por el camino X o por el camino Y?

**Limitación 3: Sin persistencia entre ejecuciones**  
Si el agente falla en el paso 5 de 10, ¿debe repetir los pasos 1-4?

**Metáfora:**
- **LangChain = Tren en vías fijas:** Eficiente, predecible, pero inflexible. Solo puede ir hacia adelante en una ruta predefinida.
- **LangGraph = Red de carreteras con GPS:** Puede tomar diferentes rutas, volver atrás si hay un obstáculo, y guardar el progreso.

Veamos un ejemplo concreto donde LangChain no es suficiente.

In [ ]:
# Caso de uso: Agente traductor con validación

# Con LangChain (lineal):
# 1. Traducir texto
# 2. Validar gramática
# 3. Si falla validación → ¿Cómo volver al paso 1?

# Problema: LangChain NO puede hacer esto fácilmente.
# Necesitamos CICLOS y DECISIONES DINÁMICAS.

print("Flujo deseado:")
print("""\n
    [Traducir] → [Validar]
         ↑           |
         |_____Si falla____|
         
    Si la validación falla, vuelve a traducir.
""")

### MICRO-RETO 1: Identifica el Problema

Para cada caso de uso, decide si necesitas **LangChain (cadena)** o **LangGraph (grafo)**.

**Criterio:**
- Si el flujo es **lineal y simple** → LangChain
- Si necesitas **ciclos, decisiones dinámicas, o persistencia** → LangGraph

**Casos:**
1. Sistema que resume un texto largo
2. Agente de soporte que clasifica ticket, busca en base de conocimiento, y si no encuentra respuesta deriva a humano
3. Generador de código que escribe código, lo ejecuta, y si falla reintenta con el error como contexto
4. Pipeline que extrae datos de PDF y los guarda en base de datos

In [ ]:
# TODO: Completa tu análisis
casos = {
    "Caso 1 (Resumen)": "?",  # LangChain o LangGraph
    "Caso 2 (Soporte)": "?",
    "Caso 3 (Generador código)": "?",
    "Caso 4 (Pipeline ETL)": "?"
}

# Justifica tu respuesta en el chat o hablando
# Caso 1: _____ porque _____

### 1.2 LangGraph: De Cadenas a Grafos

**Definición formal:**  
LangGraph es un framework para construir aplicaciones **stateful** (con estado) y **multi-actor** usando **grafos dirigidos**.

**En términos simples:**  
LangGraph te permite definir un conjunto de **nodos** (funciones) conectados por **aristas** (edges), donde cada nodo puede:
- Llamar a un LLM
- Ejecutar código arbitrario
- Llamar APIs externas
- Modificar el estado compartido
- Decidir qué nodo ejecutar siguiente

**Diferencias clave:**

| Característica | LangChain | LangGraph |
|----------------|-----------|----------|
| **Estructura** | Cadena lineal (A→B→C) | Grafo con bifurcaciones y ciclos |
| **Flujo** | Secuencial, predefinido | Dinámico, condicional |
| **Estado** | Pasa entre pasos | Estado compartido persistente |
| **Ciclos** | No soporta | Soporta ciclos controlados |
| **Persistencia** | No nativa | Checkpoints automáticos |
| **Cuándo usar** | Tareas simples, pipelines ETL | Agentes complejos, multi-paso |

**Visualización:**

```
LangChain (Chain):
    A → B → C → D
    
LangGraph (Graph):
    
        A
       / \
      B   C
       \ /
        D
        |
        E → (puede volver a B si es necesario)
```

Veamos la diferencia en código.

In [ ]:
# COMPARACIÓN: Chain vs Graph

# ========================================
# Enfoque LangChain (simulado)
# ========================================
def chain_langchain():
    # Paso 1: Procesar input
    resultado = procesar_input("Hola mundo")
    # Paso 2: Transformar
    resultado = transformar(resultado)
    # Paso 3: Output
    return generar_output(resultado)

# Problema: ¿Cómo vuelves al paso 1 si el paso 2 falla?

# ========================================
# Enfoque LangGraph
# ========================================
from typing import TypedDict

class Estado(TypedDict):
    input: str
    resultado: str
    intentos: int

def nodo_procesar(state: Estado) -> dict:
    print(f"Procesando... (intento {state['intentos']})")
    return {"resultado": f"Procesado: {state['input']}"}

def nodo_validar(state: Estado) -> dict:
    # Simula validación que puede fallar
    if state["intentos"] < 2:
        print("Validación falló, reintentando...")
        return {"intentos": state["intentos"] + 1}
    else:
        print("Validación exitosa")
        return {"resultado": "✓ Validado"}

# Con LangGraph podemos definir:
# procesar → validar → (si falla) volver a procesar
# (Lo implementaremos completo más adelante)

print("Con LangGraph podemos implementar CICLOS y DECISIONES fácilmente.")

### MICRO-RETO 2: Convierte Chain en Graph

Tienes una **chain de LangChain** de 3 pasos:
```
Paso 1: Extraer keywords de un texto
Paso 2: Buscar documentos relacionados
Paso 3: Generar resumen
```

**Tarea:**
1. Dibuja cómo se vería este flujo como **grafo** (en papel o usando texto ASCII)
2. Identifica qué debería contener el **Estado** compartido
3. Escribe los nombres de los 3 nodos como funciones Python (solo la firma, no el código completo)

In [ ]:
# TODO: Diseña el grafo

# 1. Diagrama ASCII del grafo:
# (Usa comentarios para dibujarlo)
"""
     [____?____]
          |
          v
     [____?____]
          |
          v
     [____?____]
"""

# 2. Estado compartido (TypedDict):
from typing import TypedDict

class EstadoRAG(TypedDict):
    # TODO: Añade los campos necesarios
    pass

# 3. Firmas de los nodos:
def nodo_extraer_keywords(state: EstadoRAG) -> dict:
    # TODO: Define qué devuelve este nodo
    pass

def nodo_buscar_documentos(state: EstadoRAG) -> dict:
    pass

def nodo_generar_resumen(state: EstadoRAG) -> dict:
    pass

### 1.3 Teoría de Grafos: Crash Course

Para entender LangGraph, necesitas entender qué es un **grafo** en términos de ciencia de la computación.

**Definición:**  
Un grafo es una estructura compuesta por:
- **Nodos (vértices):** Entidades o estados
- **Aristas (edges):** Conexiones entre nodos

**Tipos de grafos:**

**1. Grafo No Dirigido:**
```
A ←→ B
```
La conexión es bidireccional (redes sociales: amistad)

**2. Grafo Dirigido (LangGraph usa este):**
```
A → B
```
La conexión tiene dirección (flujo de datos)

**3. Grafo Acíclico (DAG - Directed Acyclic Graph):**
```
A → B → C → D
```
No puede volver atrás (como LangChain)

**4. Grafo Cíclico (LangGraph permite estos):**
```
A → B → C
    ↑___↓
```
Puede volver a nodos anteriores

**¿Por qué los agentes necesitan CICLOS?**

Muchas tareas del mundo real requieren **iteración y refinamiento:**
- Escribir código → Ejecutar → Si falla, revisar y reescribir
- Generar respuesta → Validar → Si es incorrecta, regenerar
- Buscar información → Evaluar → Si es insuficiente, buscar más

**Advertencia:** Los ciclos deben tener **condición de salida** clara. Un ciclo sin salida = bucle infinito.

In [ ]:
# Visualización de tipos de grafos

print("GRAFO ACÍCLICO (DAG) - Como LangChain:")
print("""\n
    START → A → B → C → END
""")

print("\nGRAFO CÍCLICO - Como LangGraph:")
print("""\n
    START → A → B → C → END
            ↑_____↓
            (ciclo controlado)
""")

print("\nGRAFO CON BIFURCACIÓN - LangGraph:")
print("""\n
             ┌→ B1 →┐
    START → A        D → END
             └→ B2 →┘
    
    El nodo A decide si ir a B1 o B2
""")

**🌍 Caso Real: Por qué GitHub Copilot Workspace necesita ciclos**

GitHub Copilot Workspace (agente que planifica y ejecuta cambios en código) tiene este flujo:

```
[Analizar issue] → [Planificar cambios] → [Generar código] 
                           ↑                      |
                           |_____Si tests fallan___|
```

Sin ciclos, el agente solo podría intentar UNA VEZ. Con ciclos, puede:
- Generar código
- Ejecutar tests
- Si fallan, volver a planificar con el error
- Reintentar hasta 5 veces

Esto es imposible con LangChain (cadenas lineales). Por eso estos agentes están construidos con LangGraph.

### MICRO-RETO 3: Diseña tu Grafo en Papel

**Escenario:** Agente de atención al cliente

**Flujo deseado:**
1. Saluda al usuario
2. Clasifica el tipo de consulta (técnica, facturación, general)
3. Dependiendo de la clasificación:
   - Si es técnica → Deriva a soporte técnico
   - Si es facturación → Deriva a contabilidad
   - Si es general → Responde directamente
4. Genera respuesta final

**Tarea:**  
Dibuja el grafo en papel o en comentarios usando texto ASCII. Identifica:
- Nodos necesarios
- Conexiones entre nodos
- Dónde hay bifurcaciones (decisiones)
- Punto de entrada y salida

In [ ]:
# TODO: Dibuja tu grafo aquí
"""
Grafo del Agente de Atención al Cliente:

[Escribe tu diagrama aquí usando caracteres ASCII]

Ejemplo de formato:
    START
      |
      v
    [Nodo A]
      |
    ...

"""

# Lista de nodos identificados:
nodos = [
    # TODO: Completa
    # "nodo_saludar",
    # ...
]

# Decisiones (bifurcaciones):
decisiones = [
    # TODO: Identifica dónde el grafo se bifurca
    # "Después de clasificar, se decide ruta"
]

---
## BLOQUE 2: Anatomía de un Grafo LangGraph

### 2.1 Nodos: Funciones con Estado

**Concepto:**  
Un **nodo** es una función Python que recibe el estado actual y devuelve actualizaciones al estado.

**Firma de un nodo:**
```python
def mi_nodo(state: TypedDict) -> dict:
    # Lee datos del estado
    # Ejecuta lógica (LLM call, API, cálculo)
    # Devuelve SOLO las actualizaciones
    return {"campo_a_actualizar": nuevo_valor}
```

**Regla de oro (Principles of Building AI Agents):**  
> "Un nodo = una responsabilidad. Si hace dos cosas, son dos nodos."

**Tipos de nodos según su función:**

1. **Nodos de transformación:** Procesan datos
2. **Nodos de llamada a LLM:** Invocan modelos de lenguaje
3. **Nodos de llamada a API:** Consultan servicios externos
4. **Nodos de decisión (routers):** Determinan el siguiente nodo a ejecutar

**Importante:**  
Los nodos NO modifican el estado directamente. Devuelven un diccionario con las actualizaciones, y LangGraph se encarga de aplicarlas.

In [ ]:
from typing import TypedDict

# Definir el estado
class EstadoEjemplo(TypedDict):
    mensaje: str
    contador: int
    procesado: bool

# ========================================
# Ejemplo 1: Nodo de transformación simple
# ========================================
def nodo_transformar(state: EstadoEjemplo) -> dict:
    """Convierte el mensaje a mayúsculas."""
    mensaje_original = state["mensaje"]
    mensaje_transformado = mensaje_original.upper()
    
    print(f"Transformando: {mensaje_original} → {mensaje_transformado}")
    
    # Devuelve SOLO lo que cambia
    return {"mensaje": mensaje_transformado}

# ========================================
# Ejemplo 2: Nodo con lógica (simula LLM)
# ========================================
def nodo_analizar(state: EstadoEjemplo) -> dict:
    """Analiza el mensaje y actualiza contador."""
    mensaje = state["mensaje"]
    num_palabras = len(mensaje.split())
    
    print(f"Análisis: El mensaje tiene {num_palabras} palabras")
    
    return {
        "contador": num_palabras,
        "procesado": True
    }

# ========================================
# Ejemplo 3: Nodo que consulta API (simulado)
# ========================================
def nodo_consultar_api(state: EstadoEjemplo) -> dict:
    """Simula llamada a API externa."""
    mensaje = state["mensaje"]
    
    # Simulamos una respuesta de API
    respuesta_api = f"API procesó: {mensaje}"
    
    print(f"Llamada a API completada")
    
    return {"mensaje": respuesta_api}

# Prueba manual de un nodo
estado_inicial = {
    "mensaje": "hola mundo",
    "contador": 0,
    "procesado": False
}

print("Estado inicial:", estado_inicial)
actualizacion = nodo_transformar(estado_inicial)
print("Actualización:", actualizacion)

### MICRO-RETO 4: Crea tu Primer Nodo

**Tarea:** Implementa un nodo que procese un mensaje de bienvenida personalizado.

**Requisitos:**
1. El estado contiene: `nombre` (str) y `saludo` (str)
2. El nodo debe generar un saludo personalizado: `"¡Hola {nombre}! Bienvenido a LangGraph."`
3. El nodo debe actualizar el campo `saludo` en el estado

In [ ]:
# TODO: Define el estado
class EstadoSaludo(TypedDict):
    nombre: str
    saludo: str

# TODO: Implementa el nodo
def nodo_saludar(state: EstadoSaludo) -> dict:
    """Genera un saludo personalizado."""
    # Tu código aquí
    pass

# Prueba tu nodo
estado_prueba = {"nombre": "Ana", "saludo": ""}
# resultado = nodo_saludar(estado_prueba)
# print(resultado)

### 2.2 Edges: El Flujo entre Nodos

**Concepto:**  
Los **edges** (aristas) definen cómo fluye la ejecución de un nodo a otro.

**Tipos de edges:**

**1. Edge Incondicional (`add_edge`):**
```python
graph.add_edge("nodo_A", "nodo_B")
```
Siempre va de A a B, sin condiciones.

**2. Edge Condicional (`add_conditional_edges`):**
```python
graph.add_conditional_edges(
    "nodo_A",
    funcion_router,  # Decide a dónde ir
    {
        "opcion_1": "nodo_B",
        "opcion_2": "nodo_C"
    }
)
```
El `funcion_router` decide dinámicamente el destino.

**Patrón Router:**
```
       [Nodo A]
          |
          v
     [Router: evalúa estado]
          |
     /----+----\
    v           v
[Nodo B]    [Nodo C]
```

**¿Cuándo usar cada uno?**
- **Incondicional:** Flujo siempre igual (ej: preparar → procesar → guardar)
- **Condicional:** Decisión dinámica (ej: clasificar → si urgente: escalar, si normal: responder)

In [ ]:
from langgraph.graph import StateGraph, END

# ========================================
# Ejemplo: Edge Incondicional
# ========================================

class EstadoSimple(TypedDict):
    valor: int

def nodo_A(state: EstadoSimple) -> dict:
    print("Ejecutando Nodo A")
    return {"valor": state["valor"] + 1}

def nodo_B(state: EstadoSimple) -> dict:
    print("Ejecutando Nodo B")
    return {"valor": state["valor"] * 2}

# Construir grafo
workflow_incondicional = StateGraph(EstadoSimple)
workflow_incondicional.add_node("A", nodo_A)
workflow_incondicional.add_node("B", nodo_B)

# Edge incondicional: SIEMPRE va de A a B
workflow_incondicional.add_edge("A", "B")
workflow_incondicional.add_edge("B", END)

workflow_incondicional.set_entry_point("A")

app_incondicional = workflow_incondicional.compile()

# Ejecutar
resultado = app_incondicional.invoke({"valor": 5})
print(f"\nResultado: {resultado}")
# 5 → A: +1 = 6 → B: *2 = 12

In [ ]:
# ========================================
# Ejemplo: Edge Condicional (con Router)
# ========================================

class EstadoCondicional(TypedDict):
    numero: int
    resultado: str

def nodo_evaluar(state: EstadoCondicional) -> dict:
    print(f"Evaluando número: {state['numero']}")
    return {}  # No modifica estado, solo evalúa

def nodo_par(state: EstadoCondicional) -> dict:
    print("El número es PAR")
    return {"resultado": "par"}

def nodo_impar(state: EstadoCondicional) -> dict:
    print("El número es IMPAR")
    return {"resultado": "impar"}

# Función router: decide el camino
def router_par_impar(state: EstadoCondicional) -> str:
    """Decide si ir al nodo 'par' o 'impar'."""
    if state["numero"] % 2 == 0:
        return "par"
    else:
        return "impar"

# Construir grafo
workflow_condicional = StateGraph(EstadoCondicional)
workflow_condicional.add_node("evaluar", nodo_evaluar)
workflow_condicional.add_node("par", nodo_par)
workflow_condicional.add_node("impar", nodo_impar)

# Edge condicional: el router decide
workflow_condicional.add_conditional_edges(
    "evaluar",
    router_par_impar,
    {
        "par": "par",
        "impar": "impar"
    }
)

workflow_condicional.add_edge("par", END)
workflow_condicional.add_edge("impar", END)

workflow_condicional.set_entry_point("evaluar")

app_condicional = workflow_condicional.compile()

# Pruebas
print("\n--- Prueba 1: número par ---")
resultado1 = app_condicional.invoke({"numero": 4, "resultado": ""})
print(f"Resultado: {resultado1}")

print("\n--- Prueba 2: número impar ---")
resultado2 = app_condicional.invoke({"numero": 7, "resultado": ""})
print(f"Resultado: {resultado2}")

### MICRO-RETO 5: Conecta los Nodos

**Escenario:** Sistema de clasificación de correos

Tienes 3 nodos definidos:
- `nodo_leer`: Lee el correo
- `nodo_spam`: Marca como spam
- `nodo_inbox`: Mueve a bandeja de entrada

**Tarea:**
1. Conecta los nodos apropiadamente
2. Implementa un router que decida si es spam o no
3. Construye y compila el grafo

In [ ]:
# Estado
class EstadoCorreo(TypedDict):
    asunto: str
    es_spam: bool
    destino: str

# Nodos ya definidos
def nodo_leer(state: EstadoCorreo) -> dict:
    print(f"Leyendo correo: {state['asunto']}")
    # Simula detección de spam (palabras clave)
    es_spam = "gratis" in state["asunto"].lower() or "ganaste" in state["asunto"].lower()
    return {"es_spam": es_spam}

def nodo_spam(state: EstadoCorreo) -> dict:
    print("→ Marcado como SPAM")
    return {"destino": "spam"}

def nodo_inbox(state: EstadoCorreo) -> dict:
    print("→ Movido a INBOX")
    return {"destino": "inbox"}

# TODO: Implementa el router
def router_correo(state: EstadoCorreo) -> str:
    """Decide si va a 'spam' o 'inbox'."""
    # Tu código aquí
    pass

# TODO: Construye el grafo
workflow_correo = StateGraph(EstadoCorreo)

# Añadir nodos
# workflow_correo.add_node(...)

# Conectar edges
# workflow_correo.add_conditional_edges(...)

# Compilar
# app_correo = workflow_correo.compile()

# Prueba
# resultado = app_correo.invoke({"asunto": "¡GANASTE UN MILLÓN!", "es_spam": False, "destino": ""})
# print(resultado)

### 2.3 Estado: La Memoria del Grafo

**Concepto:**  
El **Estado** (State) es un diccionario compartido que viaja por todos los nodos del grafo.

**Analogía (Principles of Building AI Agents, Cap 7):**
- **State = RAM del agente:** Memoria rápida, temporal, activa durante la ejecución
- **Database = Disco duro:** Almacenamiento permanente entre ejecuciones

**Otra analogía:**  
El State es como un **post-it** que pasas de persona a persona en una reunión. Cada persona (nodo) puede:
- Leer lo que hay en el post-it
- Añadir información nueva
- Modificar información existente

**¿Por qué TypedDict?**

Definir el estado con `TypedDict` nos da:
1. **Autocompletado** en el IDE
2. **Type checking** (detectar errores antes de ejecutar)
3. **Documentación** clara de qué contiene el estado

**Cómo se actualiza el estado:**

1. Los nodos devuelven un `dict` con las actualizaciones
2. LangGraph **combina** (merge) las actualizaciones con el estado actual
3. El estado actualizado pasa al siguiente nodo

In [ ]:
# Evolución del estado paso a paso

from typing import TypedDict

class EstadoPedido(TypedDict):
    cliente: str
    producto: str
    precio: float
    descuento: float
    total: float

# Estado inicial
estado = {
    "cliente": "María",
    "producto": "Laptop",
    "precio": 1000.0,
    "descuento": 0.0,
    "total": 0.0
}

print("Estado inicial:")
print(estado)
print()

# Nodo 1: Aplicar descuento
def nodo_aplicar_descuento(state: EstadoPedido) -> dict:
    descuento = state["precio"] * 0.10  # 10% descuento
    print(f"Aplicando descuento: ${descuento}")
    return {"descuento": descuento}

actualizacion_1 = nodo_aplicar_descuento(estado)
estado.update(actualizacion_1)  # LangGraph hace esto automáticamente

print("Estado después del nodo 1:")
print(estado)
print()

# Nodo 2: Calcular total
def nodo_calcular_total(state: EstadoPedido) -> dict:
    total = state["precio"] - state["descuento"]
    print(f"Calculando total: ${total}")
    return {"total": total}

actualizacion_2 = nodo_calcular_total(estado)
estado.update(actualizacion_2)

print("Estado final:")
print(estado)

print("\n✓ El estado evolucionó a través de los nodos")

#### Visualización del Grafo Condicional

Veamos cómo se ve este grafo visualmente:

#### Visualización del Grafo Incondicional

Veamos cómo se ve este grafo visualmente:

In [ ]:
# Visualización del grafo
from IPython.display import Image, display

# Opción 1: Diagrama Mermaid (visual)
try:
    display(Image(app_incondicional.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"No se pudo generar imagen: {{e}}")
    print("Mostrando versión ASCII:\n")
    # Opción 2: ASCII art (siempre funciona)
    app_incondicional.get_graph().print_ascii()

In [ ]:
# Visualización del grafo
from IPython.display import Image, display

# Opción 1: Diagrama Mermaid (visual)
try:
    display(Image(app_condicional.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"No se pudo generar imagen: {{e}}")
    print("Mostrando versión ASCII:\n")
    # Opción 2: ASCII art (siempre funciona)
    app_condicional.get_graph().print_ascii()

### MICRO-RETO 6: Rastrea el Estado

**Tarea:** Predice el estado final después de ejecutar este grafo.

**Estado inicial:**
```python
{
    "texto": "hola",
    "longitud": 0,
    "mayusculas": False
}
```

**Grafo:**
```
nodo_contar → nodo_transformar → END
```

**Nodos:**
- `nodo_contar`: Calcula longitud del texto → `{"longitud": len(state["texto"])}`
- `nodo_transformar`: Convierte a mayúsculas → `{"texto": state["texto"].upper(), "mayusculas": True}`

**Pregunta:** ¿Cuál será el estado final?

In [ ]:
# TODO: Predice el estado final sin ejecutar

estado_predicho = {
    "texto": "?",
    "longitud": "?",
    "mayusculas": "?"
}

print("Tu predicción:", estado_predicho)

# Descomenta para verificar
# class EstadoTexto(TypedDict):
#     texto: str
#     longitud: int
#     mayusculas: bool
#
# def nodo_contar(state: EstadoTexto) -> dict:
#     return {"longitud": len(state["texto"])}
#
# def nodo_transformar(state: EstadoTexto) -> dict:
#     return {"texto": state["texto"].upper(), "mayusculas": True}
#
# workflow = StateGraph(EstadoTexto)
# workflow.add_node("contar", nodo_contar)
# workflow.add_node("transformar", nodo_transformar)
# workflow.set_entry_point("contar")
# workflow.add_edge("contar", "transformar")
# workflow.add_edge("transformar", END)
# app = workflow.compile()
#
# resultado_real = app.invoke({"texto": "hola", "longitud": 0, "mayusculas": False})
# print("\nEstado real:", resultado_real)

---
## BLOQUE 3: Construyendo el Primer Grafo Completo

### 3.1 Entry Point y Compilación

Ya has visto nodos, edges y estado. Ahora vamos a unir todo para crear un grafo funcional.

**Pasos para construir un grafo:**

1. **Definir el Estado** (TypedDict)
2. **Crear el grafo** (`StateGraph`)
3. **Añadir nodos** (`add_node`)
4. **Conectar edges** (`add_edge` o `add_conditional_edges`)
5. **Definir punto de entrada** (`set_entry_point`)
6. **Compilar** (`compile()`)
7. **Ejecutar** (`invoke()`)

**Entry Point:**  
El punto de entrada es el primer nodo que se ejecuta cuando invocas el grafo.

```python
workflow.set_entry_point("nombre_del_primer_nodo")
```

**Compilación:**  
El método `compile()` valida el grafo y lo prepara para ejecución.

```python
app = workflow.compile()
```

**Ejecución:**  
```python
resultado = app.invoke(estado_inicial)
```

In [ ]:
from typing import TypedDict
from langgraph.graph import StateGraph, END

# ========================================
# Ejemplo: Grafo Completo Mínimo
# ========================================

# 1. Definir estado
class EstadoMinimo(TypedDict):
    mensaje: str
    procesado: bool

# 2. Definir nodos
def nodo_inicio(state: EstadoMinimo) -> dict:
    print("→ Nodo Inicio")
    return {"mensaje": f"Procesando: {state['mensaje']}"}

def nodo_final(state: EstadoMinimo) -> dict:
    print("→ Nodo Final")
    return {"procesado": True}

# 3. Crear grafo
workflow = StateGraph(EstadoMinimo)

# 4. Añadir nodos
workflow.add_node("inicio", nodo_inicio)
workflow.add_node("final", nodo_final)

# 5. Conectar edges
workflow.add_edge("inicio", "final")
workflow.add_edge("final", END)

# 6. Entry point
workflow.set_entry_point("inicio")

# 7. Compilar
app_minimo = workflow.compile()

print("Grafo compilado exitosamente\n")

# 8. Ejecutar
resultado = app_minimo.invoke({"mensaje": "Hola LangGraph", "procesado": False})
print(f"\nResultado final: {resultado}")

### MICRO-RETO 7: Completa el Grafo

**Escenario:** Procesador de texto con 3 etapas

Tienes el siguiente código con **huecos**:
1. Falta definir el punto de entrada
2. Falta conectar el nodo `validar` con `END`
3. Falta compilar el grafo

**Completa el código para que funcione.**

In [ ]:
# Estado
class EstadoProcesador(TypedDict):
    texto: str
    limpio: bool
    validado: bool

# Nodos
def nodo_limpiar(state: EstadoProcesador) -> dict:
    texto_limpio = state["texto"].strip().lower()
    print(f"Limpiando: '{state['texto']}' → '{texto_limpio}'")
    return {"texto": texto_limpio, "limpio": True}

def nodo_validar(state: EstadoProcesador) -> dict:
    valido = len(state["texto"]) > 0
    print(f"Validando: {'✓ Válido' if valido else '✗ Inválido'}")
    return {"validado": valido}

# Construir grafo
workflow_procesador = StateGraph(EstadoProcesador)

workflow_procesador.add_node("limpiar", nodo_limpiar)
workflow_procesador.add_node("validar", nodo_validar)

workflow_procesador.add_edge("limpiar", "validar")

# TODO: Falta 1 - Conectar 'validar' con END
# workflow_procesador.add_edge(...)

# TODO: Falta 2 - Definir punto de entrada
# workflow_procesador.set_entry_point(...)

# TODO: Falta 3 - Compilar
# app_procesador = workflow_procesador.compile()

# Prueba (descomenta después de completar)
# resultado = app_procesador.invoke({"texto": "  Hola Mundo  ", "limpio": False, "validado": False})
# print(f"\nResultado: {resultado}")

#### Visualización del Grafo Mínimo

Veamos cómo se ve este grafo visualmente:

In [ ]:
# Visualización del grafo
from IPython.display import Image, display

# Opción 1: Diagrama Mermaid (visual)
try:
    display(Image(app_minimo.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"No se pudo generar imagen: {{e}}")
    print("Mostrando versión ASCII:\n")
    # Opción 2: ASCII art (siempre funciona)
    app_minimo.get_graph().print_ascii()

### 3.2 Ejecución y Debugging

Una vez compilado el grafo, puedes ejecutarlo con `invoke()`:

```python
resultado = app.invoke(estado_inicial)
```

**Leer el output:**

El resultado es el **estado final** después de que todos los nodos terminaron de ejecutarse.

```python
{
    "campo_1": valor_final,
    "campo_2": valor_final,
    ...
}
```

**Errores comunes:**

1. **KeyError:** El nodo intenta acceder a un campo que no existe en el estado
   - Solución: Verifica que el TypedDict tenga todos los campos necesarios

2. **Node not found:** Intentas conectar un edge a un nodo que no existe
   - Solución: Verifica que el nombre del nodo en `add_edge` coincida con `add_node`

3. **No entry point:** Olvidaste definir el punto de entrada
   - Solución: Añade `workflow.set_entry_point("nombre_nodo")`

4. **Dead end:** Un nodo no tiene edge de salida
   - Solución: Todos los nodos (excepto el último) deben tener un edge hacia otro nodo o END

In [ ]:
# Ejemplo de debugging

# ========================================
# Grafo con error común (KeyError)
# ========================================

class EstadoIncompleto(TypedDict):
    entrada: str
    # Falta definir 'salida'

def nodo_con_error(state: EstadoIncompleto) -> dict:
    # Este nodo intenta acceder a 'salida' que no existe
    # return {"salida": f"Procesado: {state['entrada']}"}
    
    # Corrección: No acceder a campos no definidos
    return {"entrada": f"Procesado: {state['entrada']}"}

print("Tip: Siempre define TODOS los campos en el TypedDict")
print("Si un nodo va a modificar un campo, ese campo debe existir en el Estado")

### MICRO-RETO 8: Depura este Grafo Roto

El siguiente grafo tiene **2 errores**. Encuéntralos y corrígelos.

**Pistas:**
1. Revisa los edges (conexiones)
2. Revisa el estado (campos definidos vs usados)

In [ ]:
# Grafo con errores

class EstadoRoto(TypedDict):
    numero: int
    doble: int
    # ERROR 1: Falta un campo que se usa en nodo_cuadrado

def nodo_duplicar(state: EstadoRoto) -> dict:
    doble = state["numero"] * 2
    print(f"Duplicando: {state['numero']} → {doble}")
    return {"doble": doble}

def nodo_cuadrado(state: EstadoRoto) -> dict:
    cuadrado = state["numero"] ** 2
    print(f"Elevando al cuadrado: {state['numero']} → {cuadrado}")
    return {"cuadrado": cuadrado}  # Usa 'cuadrado' pero no está en TypedDict

# Construir grafo
workflow_roto = StateGraph(EstadoRoto)

workflow_roto.add_node("duplicar", nodo_duplicar)
workflow_roto.add_node("cuadrado", nodo_cuadrado)

workflow_roto.add_edge("duplicar", "cuadrado")
# ERROR 2: Falta conectar 'cuadrado' con END

workflow_roto.set_entry_point("duplicar")

# TODO: Corrige los 2 errores
# 1. Añade el campo 'cuadrado' al TypedDict
# 2. Conecta 'cuadrado' con END

# app_roto = workflow_roto.compile()
# resultado = app_roto.invoke({"numero": 5, "doble": 0, "cuadrado": 0})
# print(f"\nResultado: {resultado}")

### 3.3 Ciclos Controlados

Una de las características más poderosas de LangGraph es la capacidad de crear **ciclos**.

**¿Por qué necesitamos ciclos?**

Muchas tareas del mundo real requieren **intentos múltiples**:
- Generar código → Ejecutar → Si falla, reintentar
- Llamar API → Si timeout, reintentar
- Validar output → Si incorrecto, regenerar

**Advertencia crítica (Patterns for Building AI Agents, Cap 5):**
> "Todo ciclo DEBE tener una condición de salida clara. Un ciclo sin salida = bucle infinito."

**Patrón típico:**
```
[Ejecutar] → [Validar]
    ↑          |
    |__Si falla____|
    
Condición de salida:
- Máximo de intentos alcanzado
- Validación exitosa
```

In [ ]:
# Ejemplo: Grafo con ciclo controlado

class EstadoCiclo(TypedDict):
    numero: int
    intentos: int
    exito: bool

def nodo_intentar(state: EstadoCiclo) -> dict:
    intento_actual = state["intentos"] + 1
    print(f"Intento {intento_actual}...")
    
    # Simula éxito en el tercer intento
    exito = intento_actual >= 3
    
    return {
        "intentos": intento_actual,
        "exito": exito
    }

def nodo_finalizar(state: EstadoCiclo) -> dict:
    print(f"Finalizado después de {state['intentos']} intentos")
    return {}

# Router: decide si reintentar o finalizar
def router_ciclo(state: EstadoCiclo) -> str:
    if state["exito"]:
        return "finalizar"
    elif state["intentos"] >= 5:  # Límite de seguridad
        print("⚠️ Máximo de intentos alcanzado")
        return "finalizar"
    else:
        return "intentar"

# Construir grafo
workflow_ciclo = StateGraph(EstadoCiclo)

workflow_ciclo.add_node("intentar", nodo_intentar)
workflow_ciclo.add_node("finalizar", nodo_finalizar)

# Edge condicional: puede volver a 'intentar' o ir a 'finalizar'
workflow_ciclo.add_conditional_edges(
    "intentar",
    router_ciclo,
    {
        "intentar": "intentar",  # CICLO: vuelve a sí mismo
        "finalizar": "finalizar"
    }
)

workflow_ciclo.add_edge("finalizar", END)
workflow_ciclo.set_entry_point("intentar")

app_ciclo = workflow_ciclo.compile()

# Ejecutar
print("Ejecutando grafo con ciclo controlado:\n")
resultado = app_ciclo.invoke({"numero": 0, "intentos": 0, "exito": False})
print(f"\nResultado final: {resultado}")

---
### 💡 Conexión con el Mundo Real

**Lo que acabas de aprender NO es solo teoría abstracta.** Cada uno de estos conceptos es la base de agentes reales en producción:

**Ejemplo 1: Agentes de Programación (como Cursor, Windsurf)**
```
El 'nodo_intentar' que viste con el contador no es un juego.
En un agente de código real:
  - nodo_intentar = Llamada a GPT-4 para generar código Python
  - router_ciclo = Ejecuta el código y verifica si funciona
  - Si falla → Captura el error y vuelve a 'nodo_intentar' 
              con el mensaje de error como contexto
  - Si funciona → Finaliza
```

**Ejemplo 2: Agente de Atención al Cliente**
```
El 'router_correo' que clasificaba spam/inbox es el mismo patrón que:
  - Clasificar tickets de soporte (urgente/normal/bajo)
  - Derivar a diferentes equipos según el tipo
  - Si no puede resolver → Escalar a humano (otra bifurcación)
```

**Ejemplo 3: Sistema RAG con Validación**
```
¿Recuerdas el Micro-Reto 2 del pipeline RAG?
En producción, ese grafo tendría un ciclo:
  extraer_keywords → buscar_docs → generar_resumen → validar_calidad
                                        ↑________________|
  Si la calidad es baja → Vuelve a buscar con keywords ampliadas
```

**Punto clave:**  
Los nodos que parecen "simples" (contar intentos, validar booleanos) en ejemplos educativos, en la realidad son:
- Llamadas a APIs de LLMs ($$$)
- Consultas a bases de datos vectoriales
- Integraciones con servicios externos
- Lógica de negocio compleja

Pero la **estructura del grafo** es exactamente la misma que acabas de aprender. 🎯

#### Visualización del Grafo Ciclo

Veamos cómo se ve este grafo visualmente:

In [ ]:
# Visualización del grafo
from IPython.display import Image, display

# Opción 1: Diagrama Mermaid (visual)
try:
    display(Image(app_ciclo.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"No se pudo generar imagen: {{e}}")
    print("Mostrando versión ASCII:\n")
    # Opción 2: ASCII art (siempre funciona)
    app_ciclo.get_graph().print_ascii()